# AutoEncoders

AutoEncoder to predit the rating a user will give a movie

## Downloading the dataset

In [1]:
!wget "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
!unzip ml-100k.zip
!ls

--2024-02-09 14:12:07--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  1.67MB/s    in 2.8s    

2024-02-09 14:12:10 (1.67 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base

## Importing the libraries

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

## Preparing the training set and the test set


In [3]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

## Getting the number of users and movies


In [4]:
nb_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))

## Converting the data into an array with users in lines and movies in columns


In [5]:
def convert(data):
  new_data = []
  for id_users in range(1, nb_users + 1):
    id_movies = data[:, 1] [data[:, 0] == id_users]
    id_ratings = data[:, 2] [data[:, 0] == id_users]
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data
training_set = convert(training_set)
test_set = convert(test_set)

## Converting the data into Torch tensors


In [6]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

## Creating the architecture of the Neural Network


In [7]:
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

## Training the SAE


In [8]:
nb_epoch = 150
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = input.clone()
    if torch.sum(target.data > 0) > 0:
      output = sae(input)
      target.require_grad = False
      output[target == 0] = 0
      loss = criterion(output, target)
      mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
      loss.backward()
      train_loss += np.sqrt(loss.data*mean_corrector)
      s += 1.
      optimizer.step()
  print('epoch: '+str(epoch)+'loss: '+ str(train_loss/s))

epoch: 1loss: tensor(1.7723)
epoch: 2loss: tensor(1.0967)
epoch: 3loss: tensor(1.0537)
epoch: 4loss: tensor(1.0384)
epoch: 5loss: tensor(1.0308)
epoch: 6loss: tensor(1.0267)
epoch: 7loss: tensor(1.0235)
epoch: 8loss: tensor(1.0220)
epoch: 9loss: tensor(1.0208)
epoch: 10loss: tensor(1.0196)
epoch: 11loss: tensor(1.0186)
epoch: 12loss: tensor(1.0185)
epoch: 13loss: tensor(1.0179)
epoch: 14loss: tensor(1.0176)
epoch: 15loss: tensor(1.0170)
epoch: 16loss: tensor(1.0168)
epoch: 17loss: tensor(1.0168)
epoch: 18loss: tensor(1.0165)
epoch: 19loss: tensor(1.0162)
epoch: 20loss: tensor(1.0163)
epoch: 21loss: tensor(1.0157)
epoch: 22loss: tensor(1.0159)
epoch: 23loss: tensor(1.0159)
epoch: 24loss: tensor(1.0156)
epoch: 25loss: tensor(1.0157)
epoch: 26loss: tensor(1.0152)
epoch: 27loss: tensor(1.0152)
epoch: 28loss: tensor(1.0150)
epoch: 29loss: tensor(1.0140)
epoch: 30loss: tensor(1.0125)
epoch: 31loss: tensor(1.0101)
epoch: 32loss: tensor(1.0090)
epoch: 33loss: tensor(1.0063)
epoch: 34loss: tens

## Testing the SAE


In [9]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
  input = Variable(training_set[id_user]).unsqueeze(0)
  target = Variable(test_set[id_user]).unsqueeze(0)
  if torch.sum(target.data > 0) > 0:
    output = sae(input)
    target.require_grad = False
    output[target == 0] = 0
    loss = criterion(output, target)
    mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
    test_loss += np.sqrt(loss.data*mean_corrector)
    s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.9580)


In [10]:
target_user_id = 10
target_movie_id = 290
input = Variable(training_set[target_user_id-1]).unsqueeze(0)
output = sae(input)
output_numpy = output.data.numpy()
print (''+ str(output_numpy[0,target_movie_id-1]))

3.3185978
